<h1 style="margin-top: 0px; position: absolute; left: 5vh; top: -5vh;">
    How to create a<br><b>Devcontainer</b> for your<br>Python project 🐳
</h1>

Hi. Welcome everyone. So today we are going to talk about "How to create a Devcontainer for your Python project".

First: I'd like to get a show of hands 🙋‍♀️🙋‍♂️🖐:
- Who has played around with Devcontainers before?
- Who has used Devcontainers at their company?

<img src="images/presentation/jeroen-overschie-intro.png" alt="Jeroen Overschie" width="600px" />

I am Jeroen Overschie. I am a Machine Learning Engineer at GoDataDriven, working in Amsterdam. I help companies take their Machine Learning solutions into production.

## The use case

You are assigned to setup a new repo for a team. The requirements are as follows:

<img src="images/presentation/logo-spark.png" alt="Apache Spark" width="200px" class="fragment" data-fragment-index="0" style="
    display: inline-block;
    margin-right: 3%;
    margin-left: 6%;
">
<img src="images/presentation/logo-pyspark.png" alt="Apache Spark - pyspark" width="200px" class="fragment" data-fragment-index="1" style="
    display: inline-block;
    margin-right: 3%;
">
<img src="images/presentation/logo-python.png" alt="Python" width="200px" class="fragment" data-fragment-index="2" style="
    display: inline-block;
">


<!-- <table>
<tr>
<td>
<img src="images/presentation/logo-spark.png" alt="Apache Spark" width="200px" class="fragment" />
</td>
<td>
<img src="images/presentation/logo-pyspark.png" alt="Apache Spark - pyspark" width="200px" class="fragment" />
</td>
<td>
<img src="images/presentation/logo-python.png" alt="Python" width="200px" class="fragment" />
</td>
</tr>
</table> -->

So we need to align on:

- 📌 A specific Java version

- 📌 A specific Python version

- 📌 A specific `pyspark` version

→ otherwise we do not enjoy the guarantees we want in **production** code

- Apache Spark to **process data** (we have a cluster setup. Plus needs **Java**.)
- pyspark (pip package)
- Python (Python version)

→ we want **guarantees** in production code.

# “I never get to the actual work because I am always configuring Python virtual environments”

— Python Developer, source and date unknown

<img src="./images/presentation/teams-logo-64.png" 
    style="float: left; width: 32px; margin-right: 5px; position: absolute; top: -10px;"
    />
<table style="background-color: #F5F5F5">
    <tr style="background: none">
        <td style="border: none">
            <div class="teams-message-box fragment">
                <div class="teams-avatar teams-background-1">
                    PP
                </div>
                <div class="teams-message">
                    <div class="teams-name-box">
                        <span class="teams-name">Petra the Python Dev</span>
                        <time>
                            19/10 10:30
                        </time>
                    </div>
                    <!-- Hi! Sorry I couldn't join the meeting just now. I messed up my pyenv again 🥲. Wanted to fix it fast so I could at least have it working this afternoon. -->
                    Hi my venv somehow got corrupted 🥲. It's saying <code>No module named 'numpy'</code> even though I seemingly have it installed. Not sure how to fix.
                    <br>
                    P.S. this is also why I was not in the meeting.
                </div>
            </div>
        </td>
        <td style="border: none">
        </td>
        <td style="border: none">
            <div class="teams-message-box fragment">
                <div class="teams-avatar teams-background-2">
                    EO
                </div>
                <div class="teams-message">
                    <div class="teams-name-box">
                        <span class="teams-name">Erik the old project maintainer</span>
                        <time>
                            19/11 11:30
                        </time>
                    </div>
                    Hey so I was working on maintaining the old project. Just found out that some of the dependencies don't even work on my MacOS version anymore. FML
                </div>
            </div>
        </td>
    </tr>
    <tr style="background: none">
        <td style="border: none">
            <div class="teams-message-box fragment">
                <div class="teams-avatar teams-background-3">
                    BW
                </div>
                <div class="teams-message">
                    <div class="teams-name-box">
                        <span class="teams-name">Billy Windows</span>
                        <time>
                            15/11 15:30
                        </time>
                    </div>
                    I'm trying to dockerize the project but it's haaard. Everything that works on my Windows laptop seems to fail on Linux. 
                    <!-- We should have done this earlier. Anyway: I think we should ask the PO for an extension for running the entire thing in production. -->
                </div>
            </div>
        </td>
        <td style="border: none" colspan="2">
            <div class="teams-message-box fragment">
                <div class="teams-avatar teams-background-4">
                    JJ
                </div>
                <div class="teams-message">
                    <div class="teams-name-box">
                        <span class="teams-name">Johnny Junior</span>
                        <time>
                            10/10 13:32
                        </time>
                    </div>
                    Hii. Can you maybe run <code>pip show pyspark</code>? I'm curious which pyspark version you are running 🧐. Because if it works for you but not for me and also not in the CI maybe your environment is different than both. Just checking.
                </div>
            </div>
        </td>
    </tr>
    <tr style="background: none">
        <td style="border: none" colspan="3">
            <div class="teams-message-box fragment">
                <div class="teams-avatar teams-background-5">
                    JJ
                </div>
                <div class="teams-message">
                    <div class="teams-name-box">
                        <span class="teams-name">Johnny Junior</span>
                        <time>
                            19/10 09:03
                        </time>
                    </div>
                    Good day. I'm new at the company and wanted to get started working on the repo. Tried following the README steps but doesn't work. By any chance: are there any more detailed docs available for project setup? No right? 🙃
                    <!-- There weren't any docs on how to set up the correct venv right? Mine somehow got corrupted 🙃. It's saying <code>No module named 'numpy'</code> even though I seemingly have it installed. Not sure how to fix. -->
                </div>
            </div>
        </td>
    </tr>
</table>

1. PP: **Petra the Python Dev.** Hi my venv somehow got corrupted 🥲. It's saying `No module named 'numpy'`. even though I seemingly have it installed. Not sure how to fix.
P.S. this is also why I was not in the meeting.
1. EO: **Erik the old project maintainer.** Hey so I was working on maintaining the old project. Just found out that some of the dependencies don't even work on my MacOS version anymore. FML
1. BW: **Billy Windows.** I'm trying to dockerize the project but it's haaard. Everything that works on my Windows laptop seems to fail on Linux.
1. PU: **PySpark user.** Hii. Can you maybe run `pip show pyspark`? I'm curious which pyspark version you are running 🧐. Because if it works for you but not for me and also not in the CI maybe your environment is different than both. Just checking.
1. JJ: **Johnny Junior.** Good day. I'm new at the company and wanted to get started working on the repo. Tried following the README steps but doesn't work. By any chance: are there more detailed docs available for the project setup? No right? 🙃

🐛 Issues

<table style="font-size: 110%;">
    <tr style="background: none" class="fragment">
        <td style="border: none;">
            <div class="teams-avatar teams-background-1" style="top:0;">
                PP
            </div>
        </td>
        <td style="border: none; color: #5b5b5b;">
            Petra the Python Dev
        </td>
        <td style="border: none;">
            Corrupted Virtual Environment<br>
        </td>
    </tr>
    <tr style="background: none" class="fragment">
        <td style="border: none;">
            <div class="teams-avatar teams-background-2" style="top:0;">
                EO
            </div>
        </td>
        <td style="border: none; color: #5b5b5b;">
            Erik the old project maintainer
        </td>
        <td style="border: none;">
            Outdated project dependencies<br>
        </td>
    </tr>
    <tr style="background: none" class="fragment">
        <td style="border: none;">
            <div class="teams-avatar teams-background-3" style="top:0;">
                BW
            </div>
        </td>
        <td style="border: none; color: #5b5b5b;">
            Billy Windows
        </td>
        <td style="border: none;">
            Containerisation & going into production
            <br><span style="color:#ccc;">e.g. Windows / Linux / MacOS</span>
        </td>
    </tr>
    <tr style="background: none" class="fragment">
        <td style="border: none;">
            <div class="teams-avatar teams-background-4" style="top:0;">
                PU
            </div>
        </td>
        <td style="border: none; color: #5b5b5b;">
            PySpark user
        </td>
        <td style="border: none;">
            Inconsistent environments
            <br><span style="color:#ccc;">e.g. local ↔ CI ↔ prod ↔ team members</span>
        </td>
    </tr>
    <tr style="background: none" class="fragment">
        <td style="border: none;">
            <div class="teams-avatar teams-background-5" style="top:0;">
                JJ
            </div>
        </td>
        <td style="border: none; color: #5b5b5b;">
            Johnny Junior 
        </td>
        <td style="border: none;">
            No formal specification of install steps
            <br><span style="color:#ccc;">and missing docs 📚</span>
        </td>
    </tr>
</table>

<!-- <h3>How to get a ...<br> -->
<!-- 🔄 <b>reproducible</b><br>Dev environment?</h3> -->

<h3>🔄 How to get a <b>reproducible</b> Dev environment?</h3>

## Devcontainers to the rescue ⛑!

🐳 **Docker** helps us create a formal definition of our environment.

🔌 **Devcontainers** allow you to connect your editor (IDE) to that container.

<img src="./images/presentation/devcontainer-overview-4.webp" alt="Devcontainer overview" style="
    margin: 0;
">

<table>
<tr>
<td width="70%" style="vertical-align: baseline;">
<span style="color: #4d4d4d;" class="fragment">🔄 <b>Reproducible</b> means:</span>
<ul>
<li class="fragment">
⚡️ Faster onboarding
</li>
<li class="fragment">
👨‍👩‍👧‍👦 Better alignment between team members
</li>
<li class="fragment">
⏱ Smaller gap to production
</li>
</ul>
</td>
<td width="30%" style="vertical-align: top;" class="fragment">
<span style="color: #4d4d4d;">Downsides?</span>
<ul>
<li>
Docker knowledge
</li>
</ul>
</td>
</tr>
</table>

- **Docker**
- **Devcontainer**: connect to your IDE
- **Formal** instruction set for dev env setup

## 👷🏻‍♂️ Let's build a Devcontainer!

Let’s say we have a really simple project that looks like this:

```bash
$ tree .
.
├── README.md
├── requirements.txt
├── requirements-dev.txt
├── sales_analysis.py
└── test_sales_analysis.py
```

### The `.devcontainer` folder
Your Devcontainer spec will live inside the `.devcontainer` folder.

 There will be two main files:

- `devcontainer.json`
- `Dockerfile`


Create a new file called `devcontainer.json`:

```json
{
    "build": {
        "dockerfile": "Dockerfile",
        "context": ".."
    }
}
```


This does basically means: as a base for our Devcontainer, use the `Dockerfile` located in the current directory, and build it with a *current working directory* (cwd) of `..`.


So how does this `Dockerfile` look like?

<pre class="docker">
<code data-line-numbers="*|1|6|13|17">FROM python:3.10

# Install Java
RUN apt update && \
&nbsp;&nbsp;&nbsp;&nbsp;apt install -y sudo && \
&nbsp;&nbsp;&nbsp;&nbsp;sudo apt install default-jdk -y

## Pip dependencies
# Upgrade pip
RUN pip install --upgrade pip
# Install production dependencies
COPY requirements.txt /tmp/requirements.txt
RUN pip install -r /tmp/requirements.txt && \
&nbsp;&nbsp;&nbsp;&nbsp;rm /tmp/requirements.txt
# Install development dependencies
COPY requirements-dev.txt /tmp/requirements-dev.txt
RUN pip install -r /tmp/requirements-dev.txt && \
&nbsp;&nbsp;&nbsp;&nbsp;rm /tmp/requirements-dev.txt</code>
</pre>

Helping out:

<tr style="background: none" class="fragment">
    <td style="border: none;">
        <div class="teams-avatar teams-background-2" style="top:0;">
            EO
        </div>
    </td>
    <td style="border: none; color: #5b5b5b;">
        Erik the old project maintainer
    </td>
    <td style="border: none;">
        Outdated project dependencies<br>
    </td>
</tr>
<tr style="background: none" class="fragment">
    <td style="border: none;">
        <div class="teams-avatar teams-background-3" style="top:0;">
            BW
        </div>
    </td>
    <td style="border: none; color: #5b5b5b;">
        Billy Windows
    </td>
    <td style="border: none;">
        Containerisation & going into production
        <br><span style="color:#ccc;">e.g. Windows / Linux / MacOS</span>
    </td>
</tr>
<tr style="background: none" class="fragment">
    <td style="border: none;">
        <div class="teams-avatar teams-background-5" style="top:0;">
            JJ
        </div>
    </td>
    <td style="border: none; color: #5b5b5b;">
        Johnny Junior 
    </td>
    <td style="border: none;">
        No formal specification of install steps
        <br><span style="color:#ccc;">and missing docs 📚</span>
    </td>
</tr>

We are building our image on top of `python:3.10`, which is a Debian-based image. This is one of the Linux distributions that a Devcontainer can be built on. The main requirement is that **Node.js** should be able to run: VSCode automatically installs VSCode Server on the machine.

For an extensive list of supported distributions, see [“Remote Development with Linux”](https://code.visualstudio.com/docs/remote/linux).

### Opening the Devcontainer

The `.devcontainer` folder in place, now it’s time to open our Devcontainer.


Open up the command pallete (<kbd>CMD</kbd> + <kbd>Shift</kbd> + <kbd>P</kbd>) and select “*Dev Containers: Reopen in Container*”:

![Dev Containers: Reopen in Container](./images/presentation/reopen-in-devcontainer-notification.png)


Upon opening a repo with a valid `.devcontainer` folder, you are already notified:

![folder contains a dev container config file](./images/presentation/folder-contains-a-dev-container-config-file.png)


Your VSCode is now connected to the Docker container 🙌🏻:

<!-- ![VSCode is now connected to the Docker container](https://godatadriven.com/wp-content/uploads/2022/10/opening-the-devcontainer.gif) -->

<img src="./images/presentation/opening-the-devcontainer.gif" width="80%" alt="VSCode is now connected to the Docker container"/>


<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<!-- gif image -->

<span style="color: #cdcdcd;"><b>Rebuilding</b> allows you to get a fresh environment anytime you want:</span>

![Devcontainer rebuild option](images/presentation/devcontainer-vscode-rebuild.png)

Helping out:

<table style="font-size: 110%;">
    <tr style="background: none">
        <td style="border: none;">
            <div class="teams-avatar teams-background-1" style="top:0;">
                PP
            </div>
        </td>
        <td style="border: none; color: #5b5b5b;">
            Petra the Python Dev
        </td>
        <td style="border: none;">
            Corrupted Virtual Environment<br>
        </td>
    </tr>
</table>

### What is happening under the hood 🚗

<span style="color: #ccc;">Besides starting the Docker image and attaching the terminal to it, VSCode is doing a couple more things:</span>

<ol>
    <li class="fragment"><b><a href="https://code.visualstudio.com/docs/remote/vscode-server">VSCode Server</a></b> is being installed on your Devcontainer.
        <br><span style="color: #cecece;">VSCode Server is installed as a service in the container itself so your VSCode installation can communicate with the container. <br>For example, install and run <b>extensions</b>.</span>
    </li>
    <li class="fragment"><b>Config is copied</b> over.
        <br><span style="color: #cecece;">Config like <code>~/.gitconfig</code> and <code>~/.ssh/known_hosts</code> are copied over to their respective locations in the container.</span>
    </li>
    <li class="fragment"><b>Filesystem mounts</b>. 
        <br>
        <span style="color: #cecece;">VSCode automatically takes care of mounting: 
        <ul>
            <li>The folder you are running the Devcontainer from.</li>
            <li>Your VSCode workspace folder.</li>
        </ul>
        </span>
    </li>
</ol>



<span style="color: #ccc;">Besides starting the Docker image and attaching the terminal to it, VSCode is doing a couple more things:</span>

<ol>
    <li><b><a href="https://code.visualstudio.com/docs/remote/vscode-server">VSCode Server</a></b> is being installed on your Devcontainer.
        <br><span style="color: #aaa;">VSCode Server is installed as a service in the container itself so your VSCode installation can communicate with the container. <br>For example, install and run <b>extensions</b>.</span>
    </li>
    <li><b>Config is copied</b> over.
        <br><span style="color: #aaa;">Config like <code>~/.gitconfig</code> and <code>~/.ssh/known_hosts</code> are copied over to their respective locations in the container.</span>
    </li>
    <li><b>Filesystem mounts</b>. 
        <br>
        <span style="color: #aaa;">VSCode automatically takes care of mounting: 
        <ul>
            <li>The folder you are running the Devcontainer from.</li>
            <li>Your VSCode workspace folder.</li>
        </ul>
        </span>
    </li>
</ol>

### Opening your Devcontainer with the click of a button

<span style="color:#bbb;">Your entire project setup is now encapsulated in the Devcontainer. So actually we can add a <b style="color: black;">Markdown</b> button to open up the Devcontainer:</span>


<pre class="markdown"><code data-line-numbers="*|6">[
&nbsp;&nbsp;&nbsp;&nbsp;![Open in Remote - Containers](
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;https://img.shields.io/static/v1?label=Remote%20-%20Containers&message=Open&color=blue&logo=visualstudiocode
&nbsp;&nbsp;&nbsp;&nbsp;)
](
&nbsp;&nbsp;&nbsp;&nbsp;https://vscode.dev/redirect?url=vscode://ms-vscode-remote.remote-containers/cloneInVolume?url=https://github.com/godatadriven/python-devcontainer-template
)
</code></pre>


Which basically means, open this URL:

<pre class="markdown"><code data-line-numbers>vscode://
&nbsp;&nbsp;ms-vscode-remote.remote-containers/
&nbsp;&nbsp;cloneInVolume?
&nbsp;&nbsp;url=https://github.com/godatadriven/python-devcontainer-template
</code></pre>

<span style="color:#bbb;">Just modify the GitHub URL after `url=` ✓.</span>

This renders the following button:

<a href="https://vscode.dev/redirect?url=vscode://ms-vscode-remote.remote-containers/cloneInVolume?url=https://github.com/godatadriven/python-devcontainer-template"><img src="https://img.shields.io/static/v1?label=Remote%20-%20Containers&amp;message=Open&amp;color=blue&amp;logo=visualstudiocode" alt="Open in Remote - Containers" style="
    width: 400px;
"></a>

<!-- What kind of README would you rather like?

<table>
<thead>
<tr>
  <th style="text-align:center">Manual installation</th>
  <th style="text-align:center">Using a Devcontainer 🙌🏻</th>
</tr>
</thead>
<tbody>
<tr>
  <td style="text-align:center; vertical-align: top;"><img src="./images/presentation/installation-instructions-manual.png" alt=""></td>
  <td style="text-align:center;  vertical-align: top;"><img src="./images/presentation/installation-instructions-devcontainer.png" alt=""></td>
</tr>
</tbody>
</table> -->

<p>What kind of README would you rather like?</p>
<img src="./images/presentation/installation-instructions-manual.png" alt="" style="
    width: 49%;
    display: inline-block;
    margin: 0;
"><img src="./images/presentation/installation-instructions-devcontainer.png" alt="" style="
    display: inline-block;
    width: 49%;
    vertical-align: top;
    margin: 0;
">

## Extending the Devcontainer

<span style="color:#bbb;">We have built a working Devcontainer, that is great! But a couple things are still missing.</span>


- Install a **non-root user** for extra safety and good-practice

- Pass in **custom VSCode settings** and install extensions by default


- Be able to access Spark UI (**opening up port 4040**)


- Run **Continuous Integration** (CI) in the Devcontainer


Let's see how.


### Installing a non-root user

<!-- > Step 2 -->

If you `pip install` a new package, you will see the following message:

![The warning message: “*WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: [https://pip.pypa.io/warnings/venv](https://pip.pypa.io/warnings/venv)*](https://godatadriven.com/wp-content/uploads/2022/10/running-pip-as-root.png)


So let's go ahead and create a user for this scenario.

<pre class="language-bash"><code data-line-numbers="3-4"># Add non-root user
ARG USERNAME=nonroot
RUN groupadd --gid 1000 $USERNAME && \
&nbsp;&nbsp;&nbsp;&nbsp;useradd --uid 1000 --gid 1000 -m $USERNAME
## Make sure to reflect new user in PATH
ENV PATH="/home/${USERNAME}/.local/bin:${PATH}"
USER $USERNAME
</code></pre>



Add the following property to `devcontainer.json`:

```json
    "remoteUser": "nonroot"
```

That's great! When we now start the container we should connect as the user `nonroot`.


### Passing custom VSCode settings



```json
"customizations": {
    "vscode": {
        "extensions": [
            "ms-python.python"
        ],
        "settings": {
            "python.testing.pytestArgs": [
                "."
            ],
            "python.testing.unittestEnabled": false,
            "python.testing.pytestEnabled": true,
            "python.formatting.provider": "black",
            "python.linting.mypyEnabled": true,
            "python.linting.enabled": true
        }
    }
}
```

Helping out:

<tr style="background: none" class="fragment">
    <td style="border: none;">
        <div class="teams-avatar teams-background-5" style="top:0;">
            JJ
        </div>
    </td>
    <td style="border: none; color: #5b5b5b;">
        Johnny Junior 
    </td>
    <td style="border: none;">
        No formal specification of install steps
        <br><span style="color:#ccc;">and missing docs 📚</span>
    </td>
</tr>

The defined extensions are always installed in the Devcontainer. However, the defined settings provide just a **default** for the user to use, and can still be overriden by other setting scopes like: User Settings, Remote Settings or Workspace Settings.


### Accessing Spark UI

<!-- > Step 4 -->

Since we are using pyspark, it would be nice to be able to access **Spark UI**.


```json
"portsAttributes": {
    "4040": {
        "label": "SparkUI",
        "onAutoForward": "notify"
    }
},

"forwardPorts": [
    4040
]
```

When we now run our code, we get a notification we can open Spark UI in the browser:

![open Spark UI in the browser](./images/presentation/application-running-on-port-4040.png)


Resulting in the Spark UI like we know it:

<img src="./images/presentation/spark-ui-visible-in-localhost-4040.png" alt="spark UI in the browser" style="width: 70%; margin: 0;" >

### Running our CI in the Devcontainer


Wouldn't it be convenient if we could re-use our Devcontainer to run our Continuous Integration (CI) pipeline as well? Indeed, we can do this with Devcontainers. Similarly to how the Devcontainer image is built locally using `docker build`, the same can be done _within_ a CI/CD pipeline.


There are two basic options:

1. Build the Docker image _within_ the CI/CD pipeline
2. Prebuilding the image

<span style="color:#aaa">Let's see about option number (1).</span>

#### 1. Build the Docker image _within_ the CI/CD pipeline

Luckily, a GitHub Action was already setup for us to do exactly this:

[devcontainers/ci](https://github.com/devcontainers/ci)


To now build, push and run a command in the Devcontainer is as easy as:

<pre class="language-yaml"><code data-line-numbers="1|15-20|21-26">
name: Python app

on:
&nbsp;&nbsp;...

jobs:
&nbsp;build:
&nbsp;&nbsp;runs-on: ubuntu-latest

&nbsp;&nbsp;steps:
&nbsp;&nbsp;&nbsp;- name: Checkout (GitHub)
&nbsp;&nbsp;&nbsp;&nbsp;uses: actions/checkout@v3

&nbsp;&nbsp;&nbsp;- name: Login to GitHub Container Registry
&nbsp;&nbsp;&nbsp;&nbsp;uses: docker/login-action@v2
&nbsp;&nbsp;&nbsp;&nbsp;with:
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;registry: ghcr.io
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;username: ${{ github.repository_owner }}
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;password: ${{ secrets.GITHUB_TOKEN }}

&nbsp;&nbsp;&nbsp;- name: Build and run dev container task
&nbsp;&nbsp;&nbsp;&nbsp;uses: devcontainers/ci@v0.2
&nbsp;&nbsp;&nbsp;&nbsp;with:
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;imageName: ghcr.io/${{ github.repository }}/devcontainer
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;runCmd: pytest .
</code></pre>

That's great! Whenever this workflow runs on your main branch, the image will be pushed to the configured registry; in this case GitHub Container Registry (GHCR).

See below a trace of the executed GitHub Action:

<img src="./images/presentation/running-ci-in-the-devcontainer-github-actions.png" alt="running-ci-in-the-devcontainer-github-actions" style="width: 90%">

Awesome!

<span style="color: #cdcdcd;"><b>Consistent</b> environment</span>

<table style="font-size: 110%;">
    <tr style="background: none">
        <td style="border: none;">
            <div class="teams-avatar teams-background-4" style="top:0;">
                PU
            </div>
        </td>
        <td style="border: none; color: #5b5b5b;">
            PySpark user
        </td>
        <td style="border: none;">
            Inconsistent environments
            <br><span style="color:#ccc;">e.g. local ↔ CI ↔ prod ↔ team members</span>
        </td>
    </tr>
</table>

## The final Devcontainer definition

We built the following Devcontainer definitions. 


First, `devcontainer.json`:

```json
{
    "build": {
        "dockerfile": "Dockerfile",
        "context": ".."
    },

    "remoteUser": "nonroot",

    "customizations": {
        "vscode": {
            "extensions": [
                "ms-python.python"
            ],
            "settings": {
                "python.testing.pytestArgs": [
                    "."
                ],
                "python.testing.unittestEnabled": false,
                "python.testing.pytestEnabled": true,
                "python.formatting.provider": "black",
                "python.linting.mypyEnabled": true,
                "python.linting.enabled": true
            }
        }
    },
    ...
}
```

```json
"portsAttributes": {
    "4040": {
        "label": "SparkUI",
        "onAutoForward": "notify"
    }
},

"forwardPorts": [
    4040
]
```


And our `Dockerfile`:

```docker
FROM python:3.10

# Install Java
RUN apt update && \
    apt install -y sudo && \
    sudo apt install default-jdk -y

# Add non-root user
ARG USERNAME=nonroot
RUN groupadd --gid 1000 $USERNAME && \
    useradd --uid 1000 --gid 1000 -m $USERNAME
## Make sure to reflect new user in PATH
ENV PATH="/home/${USERNAME}/.local/bin:${PATH}"
USER $USERNAME

## Pip dependencies
# Upgrade pip
RUN pip install --upgrade pip
# Install production dependencies
COPY --chown=nonroot:1000 requirements.txt /tmp/requirements.txt
RUN pip install -r /tmp/requirements.txt && \
    rm /tmp/requirements.txt
# Install development dependencies
COPY --chown=nonroot:1000 requirements-dev.txt /tmp/requirements-dev.txt
RUN pip install -r /tmp/requirements-dev.txt && \
    rm /tmp/requirements-dev.txt
```

## Three environments 🎁

<span style="color:#aaa;">The above idea sets us up for having 3 different images for our entire lifecycle. One for Development, one for CI, and finally one for production.</span>



![three-environments-docker-images-devcontainer](./images/presentation/three-environments.png)

<!-- https://excalidraw.com/#json=xiyT30e-6fLScXqtXOx9-,pqW03Hw6D6FFb-0Q06VTQw -->

## Going further 🔮


- [Mounting directories ](https://code.visualstudio.com/remote/advancedcontainers/add-local-file-mount)

    💡 Pro tip: mount your AWS/GCP/Azure credentials

- Devcontainer [templates](https://containers.dev/templates)

- Devcontainer [features](https://containers.dev/features)

- **Remote Development 🌐**

    e.g. GitHub Codespaces, VM's on Azure/GCP/AWS


... and much more (see references slide)


## Concluding

🔌 **Devcontainers** connect your IDE to a running 🐳 Docker container.

→ _reproducibility_ & _isolation_ whilst getting a _native_ experience.


<table>
<tr>
<td width="70%" style="vertical-align: baseline;">
<span style="color: #4d4d4d;" class="fragment">🔄 <b>Reproducible</b> means:</span>
<ul>
<li class="fragment">
⚡️ Faster onboarding
</li>
<li class="fragment">
👨‍👩‍👧‍👦 Better alignment between team members
</li>
<li class="fragment">
⏱ Smaller gap to production
</li>
</ul>
</td>
<td width="30%" style="vertical-align: top;"  >

</td>
</tr>
</table>



Now only VSCode, but [open specification](https://containers.dev/) taking shape.


<h3>🤝</h3>
<div class="teams-avatar teams-avatar-big teams-background-1" style="top:0;left:50px; opacity:0.5;">
    PP
</div>
<div class="teams-avatar teams-avatar-big teams-background-2" style="top:0;left:50px; opacity:0.5;">
    EO
</div>
<br>
<div class="teams-avatar teams-avatar-big teams-background-3" style="top:0; opacity:0.5;">
    BW
</div>
<div class="teams-avatar teams-avatar-big teams-background-4" style="top:0; opacity:0.5;">
    PU
</div>
<div class="teams-avatar teams-avatar-big teams-background-5" style="top:0; opacity:0.5;">
    JJ
</div>

<h1>🫵</h1>


## Thanks! 🙌🏻

<div style="text-align: center;">
<img src="./images/presentation/qrcode_github_repo.png" alt="python-devcontainer-template github QR code link" width="400px" /><br>
<span style="text-align:center;">
<a href="https://github.com/godatadriven/python-devcontainer-template">github.com/godatadriven/python-devcontainer-template</a>
</span>
</div>

<!-- <table>
<tr>
<td width="40%">
<img src="./images/presentation/cat.gif" alt="moving cat gif" />
</td>
<td width="60%" style="text-align: center;">
<img src="./images/presentation/qrcode_github_repo.png" alt="python-devcontainer-template github QR code link" width="400px" /><br>
<span style="text-align:center;">
<a href="https://github.com/godatadriven/python-devcontainer-template">github.com/godatadriven/python-devcontainer-template</a>
</span>
</td>
</tr>
</table> -->

<!-- <p>What kind of README would you rather like?</p>
<img src="./images/presentation/cat.gif" alt="" style="
    width: 49%;
    display: inline-block;
    margin: 0;
"><img src="./images/presentation/installation-instructions-devcontainer.png" alt="" style="
    display: inline-block;
    width: 49%;
    vertical-align: top;
    margin: 0;
"> -->


### Awesome resources
Associated blog post:
- [How to create a Devcontainer for your Python project 🐳](https://godatadriven.com/blog/how-to-create-a-devcontainer-for-your-python-project-%F0%9F%90%B3/)

Spec:
- [containers.dev](https://containers.dev/). The official Devcontainer specification.
    - [Devcontainer templates](https://containers.dev/templates)
    - [Devcontainer features](https://containers.dev/features)



Docs:
- [Dev Containers VSCode extension](https://marketplace.visualstudio.com/items?itemName=ms-vscode-remote.remote-containers). The extension required to connect VSCode to a Devcontainer.
- [Mounting file directories](https://code.visualstudio.com/remote/advancedcontainers/add-local-file-mount) in Devcontainers.
- [Add a non-root user to a container](https://code.visualstudio.com/remote/advancedcontainers/add-nonroot-user). More explanations & instructions for adding a non-root user to your `Dockerfile` and `devcontainer.json`.
- [Pre-building dev container images](https://code.visualstudio.com/docs/remote/containers#_prebuilding-dev-container-images)


Repo's:
- [github.com/devcontainers/ci](https://github.com/devcontainers/ci). Run your CI in your Devcontainers. Built on the [Devcontainer CLI](https://github.com/devcontainers/cli).
- [github.com/devcontainers/images](https://github.com/devcontainers/images). A collection of ready-to-use Devcontainer images.
- [github.com/manekinekko/awesome-devcontainers](https://github.com/manekinekko/awesome-devcontainers). A repo pointing to yet even more awesome resources.


## About

Author: [Jeroen Overschie](https://www.github.com/dunnkers), working at [GoDataDriven](https://godatadriven.com/).